In [4]:
train_path = "data/train_data"

In [12]:
import os
import warnings
from src.utils.utils import load_data

warnings.filterwarnings("ignore")

In [31]:
import pandas as pd
train_df = load_data(train_path).sort_values("LocationCode", ascending=True)
crawl_df = pd.read_parquet("data/auto_weather_center.parquet")

In [32]:
train_df['DateTime'] = pd.to_datetime(train_df['DateTime'])
train_df['DateTime'] = train_df['DateTime'].dt.strftime("%Y-%m-%d %H:%M").to_frame()
train_df['DateTime'] = pd.to_datetime(train_df['DateTime'])
train_df['month'] = train_df['DateTime'].dt.month
train_df['day'] = train_df['DateTime'].dt.day
train_df['hour'] = train_df['DateTime'].dt.hour
train_df['unq'] = train_df['month'].astype(str) + train_df['day'].astype(str) + train_df['hour'].astype(str) 
print(train_df.columns)

Index(['LocationCode', 'DateTime', 'WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)', 'month',
       'day', 'hour', 'unq'],
      dtype='object')


In [33]:
train_df['unq'].value_counts()

unq
7116     1618
5317     1618
7117     1596
7115     1594
5316     1583
         ... 
690         2
52421       2
32821       1
52221       1
7144        1
Name: count, Length: 2957, dtype: int64

In [34]:
group = train_df.groupby(['LocationCode', 'unq'])[['WindSpeed(m/s)', 'Pressure(hpa)',
       'Temperature(°C)', 'Humidity(%)']].mean()

In [35]:
crawl_df['DateTime'] = pd.to_datetime(crawl_df['DateTime'])
crawl_df['DateTime'] = crawl_df['DateTime'].dt.strftime("%Y-%m-%d %H:%M").to_frame()
crawl_df['DateTime'] = pd.to_datetime(crawl_df['DateTime'])
crawl_df['month'] = crawl_df['DateTime'].dt.month
crawl_df['day'] = crawl_df['DateTime'].dt.day
crawl_df['hour'] = crawl_df['DateTime'].dt.hour
crawl_df['unq'] = crawl_df['month'].astype(str) + crawl_df['day'].astype(str) + crawl_df['hour'].astype(str) 


In [38]:
crawl_df['unq'].value_counts()

unq
7311     142
1215     142
3122     142
3121     142
3120     142
        ... 
11211     71
11212     71
11213     71
11214     71
42723     71
Name: count, Length: 4730, dtype: int64

In [37]:
s_group = crawl_df.groupby(['StationId', 'unq'])[['WindSpeed', 'AirPressure', 'AirTemperature', 'RelativeHumidity']].mean()



In [41]:
second_index_group = group.index.get_level_values('unq')
filtered_sgroup = s_group[
    s_group.index.get_level_values('unq').isin(second_index_group)
]

In [46]:
filtered_sgroup.columns = group.columns

In [56]:
import numpy as np
def sim(vec1, vec2):
    vec1, vec2 = vec1.to_numpy(), vec2.to_numpy()
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    return dot_product / (magnitude_vec1 * magnitude_vec2)

In [65]:
from tqdm import tqdm

unq_first_index = group.index.get_level_values('LocationCode').unique()
unq_first_index2 = filtered_sgroup.index.get_level_values('StationId').unique()
cols = len(group.columns)

pair = []
for idx in tqdm(unq_first_index):
    subgroup = group[group.index.get_level_values('LocationCode') == idx]
    max_cosine = -np.inf
    sim_station = ""
    for sta in unq_first_index2:
        subcrawl = filtered_sgroup[
            filtered_sgroup.index.get_level_values('StationId') == sta
        ]
        subgroup_reset = subgroup.reset_index(level='LocationCode')
        subcrawl_reset = subcrawl.reset_index(level='StationId')
        df_joined = pd.merge(
            subgroup_reset,
            subcrawl_reset,
            on='unq',
            how='left',
            suffixes=("_grp", "_crw")
        )
        df_joined.drop(columns=["LocationCode", "StationId"], inplace=True)
        df_joined.fillna(df_joined.mean(), inplace=True)
        cosines = 0
        for col in subgroup.columns:
            cosines += sim(df_joined[f"{col}_grp"], df_joined[f"{col}_crw"])
        cosines = cosines / cols
        if cosines > max_cosine:
            max_cosine = cosines
            sim_station = sta

    pair.append((idx, sim_station))
            

100%|██████████| 17/17 [00:10<00:00,  1.56it/s]
